<center>
<br />
<h1>Итераторы и Генераторы</h1>
<h3>Python</h3>
<br />
<h4>2021</h4> </center>

### Как устроен цикл for

In [1]:
for i in range(5):
    pass
    
for line in open('testmodule.py'):
    pass

for key in {'A' : 1, 'B' : 2, 'C' : 3}:
    pass
    
for letter in 'Hello, World':
    pass

In [2]:
dict_dir = set(dir({}))
file_dir = set(dir(open('testmodule.py')))
int_dir = set(dir(1))

dict_dir & file_dir - int_dir

{'__iter__'}

In [3]:
a = [1, 2, 3, 4]
it = a.__iter__()

it

In [4]:
set(dir(it)) - int_dir 

{'__iter__', '__length_hint__', '__next__', '__setstate__'}

In [5]:
it.__next__()

1

### Итерируемая последовательность (aka Iterable)

Это обьект у которого определён метод \_\_iter\_\_, возвращающий обьект реализующий протокол *итератора* 
(Примеры: list, dict, file, range)

### Итератор
Это обьект у которого определён метод \_\_next\_\_ (это может быть как отдельный обьект, так и, например, self самой последовательности, то есть она может быть итератором по самой себе)


Метод __next__ при каждом вызове должен возвращать следующий элемент последовательности, или выкидывать исключение  StopIteration, если последовательность кончилась



### iter и next

Определены как свободные функции, вызывающие соответствующие методы у обьектов. 

In [6]:
a = [1, 2, 3]
it = iter(a)
it

In [7]:
next(it)

1

In [8]:
lst = iter([1])
next(lst)
next(lst, 2)

2

In [9]:
def make_timer(ticks):
    def timer():
        nonlocal ticks
        ticks -= 1
        return ticks
    return timer

for t in iter(make_timer(10), 0):    # iter(function , terminal_value)
    print(t, end=' - ')

9 - 8 - 7 - 6 - 5 - 4 - 3 - 2 - 1 - 

### реализация цикла for

In [10]:
def handle(x):
    pass

In [11]:
seq = [1, 2, 3]
for x in seq:
    handle(x)

In [13]:
dir(it)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__length_hint__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
it = iter(seq)
while True:
    try:
        value = next(it)
        handle(value)
    except StopIteration:
        break

### Класс и его итератор

In [15]:
class RangeIter(object):
    def __init__(self, frm, to):
        self.to = to
        self.idx = frm    
    def __next__(self):
        if self.idx == self.to: raise StopIteration
        self.idx += 1
        return (self.idx - 1)

class Range1(object):
    def __init__(self, frm, to):
        self.to = to
        self.frm = frm
    def __iter__(self):
        return RangeIter(self.frm, self.to)
    
    
for i in Range1(2, 5):
    print(i, end=' - ') 

2 - 3 - 4 - 

### Класс -- итератор

In [16]:
class Range2(object):
    def __init__(self, frm, to):
        self.to = to
        self.idx = frm
        
    def __iter__(self):
        return self
        
    def __next__(self):
        if self.idx == self.to: raise StopIteration
        self.idx += 1
        return (self.idx - 1)

for i in Range2(2, 5):
    print(i, end=' - ')

2 - 3 - 4 - 

### Исчерпаемость

In [17]:
r1 = Range1(1, 5)
r2 = Range2(1, 5)

print(list(r1), list(r2))
print(list(r1), list(r2))


[1, 2, 3, 4] [1, 2, 3, 4]
[1, 2, 3, 4] []


### Несколько итераторов
Так как итератор является итерируемым обьектом - можно определять несколько итераторов для одного обьекта

In [18]:
class BinaryTree(object):
    def inorder(self):
        return InOrderIterator(self)

### \_\_contains__

может быть определен для итераторов

In [19]:
class object:
    def __contains__(self, value):
        for item in self:
            if item == value:
                return True
        return False

In [20]:
class Range:
    def __contains__(self, value):
        return self.frm < value < self.to

### Упрощенный протокол итерируемого - последовательность

In [21]:
class Seq(object):
    def __init__(self, lst):
        self.lst = lst
    def __len__(self):
        return len(self.lst)
    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError(idx)
        return self.lst[idx]
    
for i in Seq([1, 2, 3]):
    print(i)

1
2
3


# Генераторы

Генератор это функция исполнение которой приостанавливается, а не прекращается при возврате значения. Выполнение функции можно продолжить с того же места. 

In [22]:
def f(x):
    print('Generator enter')
    yield x
    x += 2
    yield x
    print('Generator Done')      

print('initial : type(f)', type(f))
a = f(5)
print('object created : type(a)', type(a))
print('first', next(a))
print('second', next(a))
print('third', next(a))
 

initial : type(f) <class 'function'>
object created : type(a) <class 'generator'>
Generator enter
first 5
second 7
Generator Done


StopIteration: 

In [23]:
def squares(size):
    for i in range(size):
        yield i ** 2

gen = squares(5)

next(gen)
print(list(gen))
print(list(gen))

[1, 4, 9, 16]
[]


In [24]:
def unique(seq):
    seen = set()
    for elem in seq:
        if elem not in seen:
            seen.add(elem)
            yield elem

list(unique([1, 2, 3, 1, 2, 4]))

[1, 2, 3, 4]

### Генераторы map и filter

In [25]:
def pmap(function, iterable):
    for i in iterable:
        yield function(i)

def pfilter(function, iterable):
    for i in iterable:
        if function(i):
            yield i

def pzip(*iterables):
    iters = list(pmap(iter, iterables))
    while True:
        try:
            yield [next(it) for it in iters]
        except StopIteration:
            return


In [26]:
list(pfilter(
    lambda x : not x[0] % x[1], 
    pzip(
        range(101, 200),
        range(2, 100))
    )
)

[[102, 3], [108, 9], [110, 11], [132, 33], [198, 99]]

### Генератор chain

In [27]:
def chain(*iterables):
    for iterable in iterables:
        for it in iterable:
            yield it

list(chain(range(5), [10, 20], 'test'))

[0, 1, 2, 3, 4, 10, 20, 't', 'e', 's', 't']

### Генератор enumerate

In [28]:
def enumerate(iterable):
    i = 0
    for it in iterable:
            yield i, it
            i += 1

list(enumerate('test'))

[(0, 't'), (1, 'e'), (2, 's'), (3, 't')]

### Генераторы внутри коллекций

In [29]:
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left, self.right = left, right
    
    def __iter__(self): # inorder
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value

### Выражения - генераторы

In [33]:
%timeit sum(x ** 3 for x in range(100000000) if not x % 11)   # note the absence of brackets

8.23 s ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit sum([x ** 3 for x in range(100000000) if not x % 11])

9.15 s ± 626 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### itertools :: islice

In [35]:
from itertools import islice
seq = range(10)
list(islice(seq, 2, 5))   # seq[2:5]

[2, 3, 4]

In [36]:
seq = range(10)
list(islice(seq, 1, 7, 2))   # seq[1:7:2]

[1, 3, 5]

### itertools :: count, cycle, repeat

In [37]:
from itertools import count, cycle, repeat

list(islice(cycle('test'), 2, 15))

['s', 't', 't', 'e', 's', 't', 't', 'e', 's', 't', 't', 'e', 's']

In [38]:
list(islice(repeat(42), 10))

[42, 42, 42, 42, 42, 42, 42, 42, 42, 42]

### itertools :: dropwhile and takewhile

In [39]:
from itertools import dropwhile, takewhile

list(takewhile(lambda x : x < 5, range(10)))

[0, 1, 2, 3, 4]

### itertools :: chain

In [40]:
from itertools import chain
list(chain([1, 2], 'test', range(3)))

[1, 2, 't', 'e', 's', 't', 0, 1, 2]

In [41]:
def geniter(count):
    for i in range(count):
        yield range(3)
        
list(chain.from_iterable(geniter(3)))

[0, 1, 2, 0, 1, 2, 0, 1, 2]

### itertools :: tee

In [42]:
from itertools import tee

a, b, c = tee(range(3), 3)
print(list(a), list(b), list(c))

[0, 1, 2] [0, 1, 2] [0, 1, 2]


### itertools :: комбинаторика

In [43]:
from itertools import product

list(product('AB', 'XY'))

[('A', 'X'), ('A', 'Y'), ('B', 'X'), ('B', 'Y')]

In [44]:
from itertools import permutations

list(permutations('ABC'))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

In [45]:
from itertools import combinations

list(combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

### yield as expression


In [46]:
def f(x):
    value = yield x
    print('value : {}'.format(value))
    x += 2
    value = yield x
    print('value : {}'.format(value))


a = f(5)
print('first', next(a))
print('second', next(a))
print('third', next(a))

first 5
value : None
second 7
value : None


StopIteration: 

### генераторы :: send

In [47]:
def f(x):
    value = yield x
    print('value : {}'.format(value))
    x += 2
    value = yield x
    print('value : {}'.format(value))

a = f(5)
print(next(a))
print('first', a.send(42))
print('second', a.send('Hi!'))

5
value : 42
first 7
value : Hi!


StopIteration: 

### генераторы :: throw и close

In [48]:
def f(x):
    value = 0
    while True:
        new_value = yield x + value
        value = new_value or value
        
adder = f(2)
next(adder)

adder.send(2)

4

In [49]:
adder = f(2)
next(adder)
adder.send(2)

adder.throw(TypeError)

TypeError: 

In [50]:
adder = f(2)
next(adder)
adder.send(2)

adder.close()

adder.send(2)

StopIteration: 

### Генераторы как коррутины

In [52]:
def grep(pattern):
    while True:
        line = yield 
        if pattern in line:
            print(line)
            
gen = grep('Hi')
next(gen)

gen.send('Test')
gen.send('Hi, my name is Alex!')

Hi, my name is Alex!


### yeild from
делигирует выполнение другому генератору

In [53]:
def dummy(count):
    yield from range(count)
    
def superdummy(count):
    for i in range(count):
        yield from dummy(i)

list(superdummy(5))

[0, 0, 1, 0, 1, 2, 0, 1, 2, 3]